In [1]:
import gym
import roboticsPlayroomPybullet


In [2]:
# Create an 
env = gym.make('UR5PlayAbsRPY1Obj-v0')
# Launch a GUI
env.render('human')
# Reset the env
_ = env.reset()
# Activate img observations 
env.render('playback')

current_dir=/home/sholto/Desktop/AI/RoboticsPlayroomPybullet/roboticsPlayroomPybullet/envs
current_dir=/home/sholto/Desktop/AI/RoboticsPlayroomPybullet/roboticsPlayroomPybullet/envs
current_dir=/home/sholto/Desktop/AI/RoboticsPlayroomPybullet/roboticsPlayroomPybullet/envs


/home/sholto/anaconda3/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


/home/sholto/Desktop/AI/RoboticsPlayroomPybullet/roboticsPlayroomPybullet/envs


In [4]:

import os
from pathlib import Path
import numpy as np


TEST_DATASET = "UR5_slow_gripper_test" 
print('Using local setup')
WORKING_PATH = Path().absolute().parent
print(f'Working path: {WORKING_PATH}')
os.chdir(WORKING_PATH)
STORAGE_PATH = WORKING_PATH
print(f'Storage path: {STORAGE_PATH}')
TRAIN_DATA_PATHS = [STORAGE_PATH/'data'/x for x in ["UR5", "UR5_high_transition", "UR5_slow_gripper"]]
TEST_DATA_PATH = STORAGE_PATH/'data'/TEST_DATASET

Using local setup
Working path: /home/sholto/Desktop/AI/learning_from_play
Storage path: /home/sholto/Desktop/AI/learning_from_play


In [ ]:
import scipy.misc
from IPython.display import display, clear_output
from natsort import natsorted

keys = ['obs', 'acts', 'achieved_goals']
full_state_save_interval = 30 # How often we saved the determinstically resettable state
# TRAIN_DATA_PATHS are the big folders which represent a few hours of teleop on different days
# Within each of these folders are indiidual trajectories, which are occasionally reset for breaks 
# If the block goes out of reach
# Strucure
# DIR
 # DIR/obs_act_etc/  - this contains the npz files with obs/acts/ags at 25Hz, all info necessary to train models
                # /1
                # /2 etc
 # DIR/states_and_ims/ - this contains the states we can reset to (which contain info like precise contact points\
                        # which is better for deterministic reset and rollout, but large so we only saved these at 1Hz
for DIR in TRAIN_DATA_PATHS:
    DIR = str(DIR) # glob/natsorted prefer strings
    
    obs_act_path = DIR+'/obs_act_etc/'

    for demo in natsorted(os.listdir(obs_act_path)):
        # Gets the saved states (which we can reset to) 
        start_points = natsorted(glob.glob(DIR+'/states_and_ims/'+str(demo)+'/env_states/*.bullet'))
        # The actual data
        traj = np.load(obs_act_path+demo+'/data.npz')
        d = {k:traj[k] for k in keys}
        # The actions taken
        acts = d['acts']
        # The total length of this recorded trajectory
        set_len = len(acts)
    
        start = 0 
        # End is the steps till the next reset state, or the full length of the seq
        end= min(start+full_state_save_interval, set_len)
        print(DIR+'/states_and_ims/'+str(demo)+'/ims')

        # Between each state we reset to - roll out the actions. 
        # A tiny amount of divergence is to be expected, which is why we reset whenever we have the information to do so
        for start_point in start_points:
            env.p.restoreState(fileName=start_point)
            env.instance.updateToggles() # need to do it when when you restore from a state as colors are not saved
            for i in range(start, end):
                # Play out the teleoperated actions
                o,r,_,_ = env.step(acts[i])

            start += full_state_save_interval
            end = min(start+full_state_save_interval, set_len)

/home/sholto/Desktop/AI/learning_from_play/data/UR5/states_and_ims/0/ims
